Section 2 — Richardson Extrapolation with Application to Numerical Derivatives. 

**Richardson Extrapolation with Application to Numerical Derivatives**

Originally devised by L.\,F.\ Richardson for weather prediction, Richardson extrapolation is a general technique to eliminate leading error terms. Here we apply it to obtain higher-accuracy numerical differentiation schemes. 


**Central-difference error structure**

Starting from the three-point central difference,
\begin{equation}
f'(x) \;=\; \frac{f(x+h)-f(x-h)}{2h}\;-\;\frac{1}{6}h^{2}f^{(3)}(x)\;+\;\cdots \tag{2.1} % 
\end{equation}
and using the same formula at step size $2h$,
\begin{equation}
f'(x) \;=\; \frac{f(x+2h)-f(x-2h)}{4h}\;-\;\frac{4}{6}h^{2}f^{(3)}(x)\;+\;\cdots \tag{2.2} % 
\end{equation}
we note that the leading $O(h^{2})$ error in \eqref{2.1} is four times smaller than in \eqref{2.2}.

Subtracting $\frac{1}{4} \times$ equation 2.2 from equation 2.1 eliminates the $h^{2}$ term and yields the five-point central difference:
\begin{equation}
f'(x) \;=\; \frac{\,f(x-2h)-f(x+2h)+8f(x+h)-8f(x-h)\,}{12h}
\;+\;\frac{1}{30}h^{4}f^{(5)}(x)\;+\;\cdots \tag{2.3} % 
\end{equation}

\
*Recursive Richardson table for derivatives*

Define the shorthand
\begin{equation}
D_1(h) \;:=\; \frac{f(x+h)-f(x-h)}{2h}, 
\qquad
D_1(2h) \;:=\; \frac{f(x+2h)-f(x-2h)}{4h}, \tag{2.4--2.5} % 
\end{equation}
so that $D_1$ has even-power error terms with leading $O(h^{2})$.

One step of Richardson extrapolation removes that leading term:
\begin{equation}
D_2(h) \;:=\; D_1(h)\;-\;\frac{D_1(2h)-D_1(h)}{2^{2}-1}
\;=\;\frac{4D_1(h)-D_1(2h)}{3}, \tag{2.6} % 
\end{equation}
leaving an $O(h^{4})$ error.

Repeating the idea on $D_2$ removes the $O(h^{4})$ term:
\begin{equation}
D_3(h) \;:=\; D_2(h)\;-\;\frac{D_2(2h)-D_2(h)}{2^{4}-1}
\;=\;\frac{16D_2(h)-D_2(2h)}{15}. \tag{2.7} % 
\end{equation}

In general, if $D_n$ has leading error $O(h^{2n})$, then
\begin{equation}
D_{n+1}(h) \;:=\; D_n(h)\;-\;\frac{D_n(2h)-D_n(h)}{2^{2n}-1}
\;=\;\frac{2^{2n}D_n(h)-D_n(2h)}{2^{2n}-1}. \tag{2.8} % 
\end{equation}

*Computational layout*

A convenient triangular table computes higher columns using previously computed entries; only one new base evaluation $D_1$ per row is needed. Four or five columns typically suffice before round-off dominates. 

\begin{center}
\renewcommand{\arraystretch}{1.3}
\begin{tabular}{lcccc}
$h$ & $D_1$ & $D_2$ & $D_3$ & $\cdots$ \\
\hline
$0.4$ & $D_1(0.4)$ & & & \\
$0.2$ & $D_1(0.2)$ & $\displaystyle \frac{4D_1(0.2)-D_1(0.4)}{3}$ & & \\
$0.1$ & $D_1(0.1)$ & $\displaystyle \frac{4D_1(0.1)-D_1(0.2)}{3}$ &
$\displaystyle \frac{16D_2(0.1)-D_2(0.2)}{15}$ & \\
$0.05$ & $D_1(0.05)$ & $\displaystyle \frac{4D_1(0.05)-D_1(0.1)}{3}$ &
$\displaystyle \frac{16D_2(0.05)-D_2(0.1)}{15}$ &
$\displaystyle \frac{64D_3(0.05)-D_3(0.1)}{63}$ \\
$\vdots$ & $\vdots$ & $\vdots$ & $\vdots$ & 
\end{tabular}
\end{center}

\noindent\textit{Practical notes.} Each new row introduces only one new $D_1(h)$ evaluation (at the halved step). Progressing across the row uses \eqref{2.8} to refine the estimate and suppress successive even-power error terms until floating-point round-off limits further improvement. 


In [2]:
import numpy as np
from dataclasses import dataclass

In [3]:
@dataclass
class RichardsonResult:
    """Holds the full Richardson table and the best estimate."""
    h0: float                 # initial step
    base_order: int           # leading error order p (e.g., 2 for central difference)
    table: list               # table[k][n] with k=extrapolation level, n=row index
    hs: np.ndarray            # step sizes used for base evaluations (h0/2**n)
    best: float               # best (most extrapolated) estimate
    best_level: int           # column index (extrapolation level) of 'best'
    best_row: int             # row index (corresponds to smallest h)

def central_difference(f, x, h):
    """Three-point central-difference approximation for f'(x). Error ~ O(h^2)."""
    return (f(x + h) - f(x - h)) / (2.0 * h)

def richardson_extrapolate_sequence(base_vals, p):
    """
    Generic Richardson extrapolation on a sequence of approximations whose leading error ~ h^p.
    Assumes entries correspond to step sizes h, h/2, h/4, ... (i.e., geometric halving).
    
    Parameters
    ----------
    base_vals : list or 1D array
        A[0,n] = base approximation at h / 2**n, n = 0..N
    p : int
        Leading error order of the base method (e.g., 2 for central difference, 2 for trapezoid).
        
    Returns
    -------
    table : list of lists
        table[k][n] is the k-th extrapolation level using rows up to n (0-indexed).
        Shape is triangular: level k has length N-k+1.
    """
    base_vals = np.asarray(base_vals, dtype=float)
    N = len(base_vals) - 1
    table = [base_vals.copy()]  # level 0

    # Build Romberg-style Richardson table:
    # A[k][n] = (2^(p*k) * A[k-1][n] - A[k-1][n-1]) / (2^(p*k) - 1)
    for k in range(1, N + 1):
        prev = table[k - 1]
        factor = 2 ** (p * k)
        curr = (factor * prev[1:] - prev[:-1]) / (factor - 1.0)
        table.append(curr)
    return table

def richardson_derivative(
    f, x, h0=0.4, rows=4, max_levels=None, tol=None
):
    """
    Compute f'(x) with Richardson extrapolation using central differences as base.
    
    Parameters
    ----------
    f : callable
        Function f(x).
    x : float
        Point at which to differentiate.
    h0 : float, optional
        Initial step size for the first row (then halved each row). Default 0.4.
    rows : int, optional
        Number of base rows (evaluations at h0/2**n). >= 2 recommended. Default 4.
    max_levels : int or None
        Max extrapolation depth (columns). If None, uses rows-1.
    tol : float or None
        If provided, the function will pick the most extrapolated value whose change from
        the previous level is < tol (using the last row). If None, it returns the most extrapolated.
    
    Returns
    -------
    RichardsonResult
    """
    if rows < 2:
        raise ValueError("rows must be >= 2")
    if max_levels is None:
        max_levels = rows - 1

    # Step sizes: h_n = h0 / 2**n
    hs = h0 / (2.0 ** np.arange(rows))

    # Base approximations (central difference) at decreasing h
    base_vals = [central_difference(f, x, h) for h in hs]

    # Perform generic Richardson extrapolation with p=2 (central difference error ~ O(h^2))
    p = 2
    full_table = richardson_extrapolate_sequence(base_vals, p)

    # Truncate to max_levels
    full_table = full_table[: max_levels + 1]  # levels 0..max_levels

    # Choose “best” estimate:
    # default: last column, last row (most extrapolated with smallest h)
    best_level = len(full_table) - 1
    best_row = len(full_table[best_level]) - 1
    best = full_table[best_level][best_row]

    # Optional tolerance-based early pick (using last row entries across levels)
    if tol is not None and best_level > 0:
        # Compare successive levels at the smallest h (i.e., last row per level)
        # Find the first level where improvement is within tol
        col_vals = [full_table[k][-1] for k in range(len(full_table))]
        for k in range(1, len(col_vals)):
            if abs(col_vals[k] - col_vals[k - 1]) < tol:
                best_level = k
                best_row = len(full_table[k]) - 1
                best = col_vals[k]
                break

    return RichardsonResult(
        h0=h0,
        base_order=p,
        table=[np.array(col) for col in full_table],
        hs=hs,
        best=best,
        best_level=best_level,
        best_row=best_row,
    )

# ---------- Example usage ----------
if __name__ == "__main__":
    # Example: f(x) = x * exp(x); f'(x) = exp(x) * (1 + x)
    def f(x):
        return x * np.exp(x)

    x0 = 2.0
    true_deriv = np.exp(x0) * (1.0 + x0)

    res = richardson_derivative(f, x0, h0=0.4, rows=6, max_levels=5, tol=1e-10)

    # Pretty-print the Richardson table
    print("Step sizes h:", res.hs)
    print("\nRichardson table (level 0 is base central-difference):")
    for k, col in enumerate(res.table):
        print(f"Level {k}: {col}")

    print("\nBest estimate:", res.best)
    print("True value   :", true_deriv)
    print("Abs error    :", abs(res.best - true_deriv))


Step sizes h: [0.4    0.2    0.1    0.05   0.025  0.0125]

Richardson table (level 0 is base central-difference):
Level 0: [23.16346429 22.41416066 22.22878688 22.18256486 22.17101693 22.16813042]
Level 1: [22.16439278 22.16699562 22.16715752 22.16716762 22.16716825]
Level 2: [22.16716914 22.16716831 22.1671683  22.1671683 ]
Level 3: [22.1671683 22.1671683 22.1671683]
Level 4: [22.1671683 22.1671683]
Level 5: [22.1671683]

Best estimate: 22.167168296792223
True value   : 22.16716829679195
Abs error    : 2.7355895326763857e-13
